# Learning regression

In [1]:
import pandas
import quandl
import math
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [3]:
quandl.ApiConfig.api_key = "your api key"
df = quandl.get("WIKI/AAPL")

In [4]:
df.to_parquet("aaplStockPrice.parquet")
print(df.head())

             Open   High    Low  Close     Volume  Ex-Dividend  Split Ratio  \
Date                                                                          
1980-12-12  28.75  28.87  28.75  28.75  2093900.0          0.0          1.0   
1980-12-15  27.38  27.38  27.25  27.25   785200.0          0.0          1.0   
1980-12-16  25.37  25.37  25.25  25.25   472000.0          0.0          1.0   
1980-12-17  25.87  26.00  25.87  25.87   385900.0          0.0          1.0   
1980-12-18  26.63  26.75  26.63  26.63   327900.0          0.0          1.0   

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
Date                                                                 
1980-12-12   0.422706   0.424470  0.422706    0.422706  117258400.0  
1980-12-15   0.402563   0.402563  0.400652    0.400652   43971200.0  
1980-12-16   0.373010   0.373010  0.371246    0.371246   26432000.0  
1980-12-17   0.380362   0.382273  0.380362    0.380362   21610400.0  
1980-12-18   0.391536   0.

Looking at adjusted prices.

In [30]:
df = df [["Adj. Open", "Adj. High", "Adj. Low", "Adj. Close", "Adj. Volume"]]
df["volHL"] = (df["Adj. High"] - df["Adj. Low"]) / df["Adj. Low"] * 100.0
df["volOC"] = (df["Adj. Close"]- df["Adj. Open"]) / df["Adj. Open"] * 100.0

df = df [["Adj. Close", "volHL", "volOC", "Adj. Volume"]]

In [31]:
print(df.head())

            Adj. Close     volHL     volOC  Adj. Volume
Date                                                   
1980-12-12    0.422706  0.417391  0.000000  117258400.0
1980-12-15    0.400652  0.477064 -0.474799   43971200.0
1980-12-16    0.371246  0.475248 -0.473000   26432000.0
1980-12-17    0.380362  0.502513  0.000000   21610400.0
1980-12-18    0.391536  0.450620  0.000000   18362400.0


Chose the column we want to predict (in this case it's the closing price) and the output. 
Next we fill Nan values to avoid removing valuable data.

In [32]:
forecastCol = "Adj. Close"
forecastOut = int(math.ceil(0.1 * len(df)))
df.fillna(-99999, inplace = True)

Adding a label column being forecast column of our choosing shifted for output we selected.

In [33]:
df["label"] = df[forecastCol].shift(-forecastOut)
df.dropna(inplace = True)

In [38]:
print(df.head())

            Adj. Close     volHL     volOC  Adj. Volume     label
Date                                                             
1980-12-12    0.422706  0.417391  0.000000  117258400.0  0.385949
1980-12-15    0.400652  0.477064 -0.474799   43971200.0  0.385949
1980-12-16    0.371246  0.475248 -0.473000   26432000.0  0.389625
1980-12-17    0.380362  0.502513  0.000000   21610400.0  0.389625
1980-12-18    0.391536  0.450620  0.000000   18362400.0  0.387713


Preparing the data set axis.

In [49]:
X = np.array(df.drop(["label"], axis = 1))
y = np.array(df["label"])
print(len(X), len(y))

8460 8460


We standardize and scale the data set with imported preprocessing function from sklearn.
<br> Docs: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html

In [50]:
X = preprocessing.scale(X)

Spliting the data set into training and testing sets having 80% for training and 20% for testing data.

In [51]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

Next we select the classifier and test the accuracy of out linear regression.

In [53]:
classifier = LinearRegression()
classifier.fit(X_train, y_train)
accuracy = classifier.score(X_test, y_test)
print(accuracy)

0.8988466523371675


The result is excellent considering we use such simple prediction algorithm.